# ファインチューニング分類の例

`babbage-002` 分類器（`ada` モデルの後継）をファインチューニングして、野球とホッケーの2つのスポーツを区別できるようにします。

In [9]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai
import os

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

## データ探索
ニュースグループデータセットはsklearnを使用して読み込むことができます。まず、データ自体を見てみましょう：

In [10]:
print(sports_dataset['data'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [11]:
sports_dataset.target_names[sports_dataset['target'][0]]


'rec.sport.baseball'

In [12]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


上記に野球カテゴリのサンプルを1つ示しています。これはメーリングリストへのメールです。合計で1197の例があり、2つのスポーツ間で均等に分割されていることが確認できます。

## データ準備
データセットをpandasデータフレームに変換し、promptとcompletionの列を作成します。promptにはメーリングリストからのメールが含まれ、completionはスポーツの名前（hockeyまたはbaseball）になります。デモンストレーション目的とファインチューニングの高速化のため、300個の例のみを使用します。実際のユースケースでは、例が多いほどパフォーマンスが向上します。

In [13]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

,prompt,completion
0,From: dougb@comm.mot.com (Doug Bank)\nSubject:...,baseball
1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
2,From: rudy@netcom.com (Rudy Wade)\nSubject: Re...,baseball
3,From: monack@helium.gas.uug.arizona.edu (david...,hockey
4,Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....,baseball


野球とホッケーの両方が単一のトークンです。データセットをjsonlファイルとして保存します。

In [6]:
df.to_json("sport2.jsonl", orient='records', lines=True)

### データ準備ツール
ファインチューニングを行う前に、データセットに対していくつかの改善を提案するデータ準備ツールを使用できます。ツールを起動する前に、最新のデータ準備ツールを使用していることを確認するためにopenaiライブラリを更新します。さらに、すべての提案を自動的に受け入れる`-q`オプションを指定します。

In [14]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

Analyzing...

- Your file contains 1197 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 11 examples that are very long. These are rows: [134, 200, 281, 320, 404, 595, 704, 838, 1113, 1139, 1174]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts e

このツールは、データセットに対していくつかの改善点を有用に提案し、データセットを訓練用と検証用に分割します。

プロンプトと完了の間にサフィックスを設けることは、入力テキストが終了したことをモデルに伝え、今度はクラスを予測する必要があることを示すために必要です。各例で同じセパレータを使用するため、モデルはセパレータの後にbaseballまたはhockeyのいずれかを予測することを学習できます。

完了部分での空白プレフィックスは有用です。なぜなら、ほとんどの単語トークンは空白プレフィックス付きでトークン化されるからです。

このツールは、これが分類タスクである可能性が高いことも認識したため、データセットを訓練用と検証用データセットに分割することを提案しました。これにより、新しいデータに対する期待されるパフォーマンスを簡単に測定できるようになります。

## ファインチューニング
このツールは、データセットを訓練するために以下のコマンドを実行することを提案しています。これは分類タスクなので、分類用途において提供された検証セットでの汎化性能がどの程度かを知りたいと思います。

CLIツールから提案されたコマンドを単純にコピーできます。特に `-m ada` を追加して、より安価で高速なadaモデルをファインチューニングします。このモデルは通常、分類用途においてより低速で高価なモデルと同等の性能を発揮します。

In [25]:
train_file = client.files.create(file=open("sport2_prepared_train.jsonl", "rb"), purpose="fine-tune")
valid_file = client.files.create(file=open("sport2_prepared_valid.jsonl", "rb"), purpose="fine-tune")

fine_tuning_job = client.fine_tuning.jobs.create(training_file=train_file.id, validation_file=valid_file.id, model="babbage-002")

print(fine_tuning_job)

FineTuningJob(id='ftjob-REo0uLpriEAm08CBRNDlPJZC', created_at=1704413736, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-9HXYFy8ux4r6aboFyec2OLRf', result_files=[], status='validating_files', trained_tokens=None, training_file='file-82XooA2AUDBAUbN5z2DuKRMs', validation_file='file-wTOcQF8vxQ0Z6fNY2GSm0z4P')


モデルは約10分で正常に訓練されます。ファインチューニングの進行状況は [https://platform.openai.com/finetune/](https://platform.openai.com/finetune/) で確認できます。

また、プログラム的にステータスを確認することもできます：

In [58]:
fine_tune_results = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
print(fine_tune_results.finished_at)

1704414393


### [上級] 結果と期待されるモデル性能
検証用に保留されたバリデーションセットでの期待される性能を観察するために、結果ファイルをダウンロードできます。

In [48]:
fine_tune_results = client.fine_tuning.jobs.retrieve(fine_tuning_job.id).result_files
result_file = client.files.retrieve(fine_tune_results[0])
content = client.files.content(result_file.id)
# save content to file
with open("result.csv", "wb") as f:
    f.write(content.text.encode("utf-8"))

In [52]:
results = pd.read_csv('result.csv')
results[results['train_accuracy'].notnull()].tail(1)

,step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
2843,2844,0.0,1.0,NaN,NaN


精度は99.6%に達します。下のプロットでは、訓練実行中に検証セットでの精度がどのように向上するかを確認できます。

In [ ]:
results[results['train_accuracy'].notnull()]['train_accuracy'].plot()

## モデルの使用
これでモデルを呼び出して予測を取得できます。

In [55]:
test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head()

,prompt,completion
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
1,From: smorris@venus.lerc.nasa.gov (Ron Morris ...,hockey
2,From: golchowy@alchemy.chem.utoronto.ca (Geral...,hockey
3,From: krattige@hpcc01.corp.hp.com (Kim Krattig...,baseball
4,From: warped@cs.montana.edu (Doug Dolven)\nSub...,baseball


ファインチューニング中に使用したのと同じセパレーターをプロンプトの後に使用する必要があります。この場合は`\n\n###\n\n`です。分類に関心があるため、temperatureをできるだけ低く設定し、モデルの予測を決定するために1トークンの補完のみが必要です。

In [69]:
ft_model = fine_tune_results.fine_tuned_model

# note that this calls the legacy completions api - https://platform.openai.com/docs/api-reference/completions
res = client.completions.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0)
res.choices[0].text


' hockey'

ログ確率を取得するには、completion リクエストで `logprobs` パラメータを指定できます。

In [71]:
res = client.completions.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].logprobs.top_logprobs

[{' hockey': 0.0, ' Hockey': -22.504879}]

モデルがベースボールよりもホッケーをはるかに高い確率で予測していることがわかり、これは正しい予測です。`log_probs`をリクエストすることで、各クラスの予測（対数）確率を確認できます。

### 汎化性能
興味深いことに、我々のファインチューニングされた分類器は非常に汎用性が高いです。異なるメーリングリストへのメールで訓練されたにも関わらず、ツイートの予測も成功しています。

In [73]:
sample_hockey_tweet = """Thank you to the 
@Canes
 and all you amazing Caniacs that have been so supportive! You guys are some of the best fans in the NHL without a doubt! Really excited to start this new chapter in my career with the 
@DetroitRedWings
 !!"""
res = client.completions.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].text

' hockey'

In [ ]:
sample_baseball_tweet="""BREAKING: The Tampa Bay Rays are finalizing a deal to acquire slugger Nelson Cruz from the Minnesota Twins, sources tell ESPN."""
res = client.completions.create(model=ft_model, prompt=sample_baseball_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].text